In [1]:
# Initialize packages
import pandas as pd
import altair as alt
import numpy as np

# Project Background

This project aims to build a model to use energy consumptions per capita to predict CO2 Emission per capita of a country.

<!-- BEGIN QUESTION -->

<div class="alert alert-info" style="color:black">

## Data Cleaning


<p>The data cleaning part includes:</p>

<ul>
  <li>Importing Data.</li>
    <ul>
      <li><a href="https://www.gapminder.org/free-material/">
          FREE DATA FROM WORLD BANK VIA GAPMINDER.ORG.</a></li>
    </ul>
  <li>Data cleaning and data wrangling using <code>pandas</code> and <code>altair</code>.</li>
</ul>
</div>

In [32]:
# read datasets and melt dataframe from wide table to long table
# co2 emission per capita
co2_e = pd.read_csv("Data/Raw/co2_emissions_tonnes_per_person.csv").melt(id_vars='country', var_name='year', value_name='co2_e')
# coal consumption per capita
coal_c = pd.read_csv("Data/Raw/coal_consumption_per_cap.csv").melt(id_vars='country', var_name='year', value_name='coal_c')
# electricity generation per capita
elec_g = pd.read_csv("Data/Raw/electricity_generation_per_person.csv").melt(id_vars='country', var_name='year', value_name='elec_g')
# electricity consumption per capita
elec_c = pd.read_csv("Data/Raw/electricity_use_per_person.csv").melt(id_vars='country', var_name='year', value_name='elec_c')
# hydro generation per capita
hydro_g = pd.read_csv("Data/Raw/hydro_power_generation_per_person.csv").melt(id_vars='country', var_name='year', value_name='hydro_g')
# nuclear generation per capita
nuclear_g = pd.read_csv("Data/Raw/nuclear_power_generation_per_person.csv").melt(id_vars='country', var_name='year', value_name='nuclear_g')
# gas generation per capita
gas_g = pd.read_csv("Data/Raw/natural_gas_production_per_person.csv").melt(id_vars='country', var_name='year', value_name='gas_g')
# oil consumption per capita
oil_c = pd.read_csv("Data/Raw/oil_consumption_per_cap.csv").melt(id_vars='country', var_name='year', value_name='oil_c')
# oil generation per capita
oil_g = pd.read_csv("Data/Raw/oil_production_per_person.csv").melt(id_vars='country', var_name='year', value_name='oil_g')
coal_c

,country,year,coal_c
0,UAE,1965,0.0
1,Argentina,1965,0.0328
2,Australia,1965,1.54
3,Austria,1965,0.696
4,Azerbaijan,1965,NaN
...,...,...,...
4340,USA,2019,0.823
4341,Uzbekistan,2019,0.0526
4342,Venezuela,2019,0.00265
4343,Vietnam,2019,0.513


In [34]:
#merging data
dataframes = [co2_e, coal_c, elec_g, elec_c, hydro_g, nuclear_g, gas_g, oil_c, oil_g]
merged_df = co2_e
for df in dataframes[1:]:
    # Merge each DataFrame with the merged DataFrame
    merged_df = pd.merge(merged_df, df, on=['country','year'], how ='outer')
merged_df

,country,year,co2_e,coal_c,elec_g,elec_c,hydro_g,nuclear_g,gas_g,oil_c,oil_g
0,Afghanistan,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UAE,1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
42803,Yemen,2019,NaN,NaN,NaN,NaN,NaN,NaN,0.017,NaN,0.145
42804,Zambia,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42805,Zimbabwe,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42806,Equatorial Guinea,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.040


In [53]:
#EDA to remove years and countries with too many NaN
cleaned_df = merged_df.query('not co2_e.isna() and not coal_c.isna() and not elec_c.isna() and not oil_c.isna()')
# remove rows that co2_e, coal_c, elec_c and oil_c do not have na value. But we think it's acceptable that elec_g, hydro_g, nuclear_g and gas_g have na value. 
# Filling NaN with 0.
cleaned_df = cleaned_df.fillna(0)
cleaned_df = cleaned_df.reset_index().drop(columns='index')
cleaned_df

,country,year,co2_e,coal_c,elec_g,elec_c,hydro_g,nuclear_g,gas_g,oil_c,oil_g
0,Australia,1965,10.70,1.54,0,2630,0.0628,0,0.000,1.340,0.0306
1,Austria,1965,5.23,0.696,0,2310,0.186,0,0.000,0.761,0.0000
2,Belgium,1965,11.20,2.03,0,2160,0.00248,0,0.000,1.690,0.0000
3,Canada,1965,12.80,0.788,0,6910,0.516,0.00056,0.000,2.740,2.2400
4,Switzerland,1965,5.22,0.219,0,3590,0.364,0,0.000,1.380,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
3340,South Africa,2014,8.86,1.64,4670,4180,0,0,0.000,0.485,0.0000
3341,Indonesia,2015,1.96,0.198,906,910.0,0,0,0.253,0.271,0.1570
3342,Indonesia,2016,2.15,0.204,948,956.0,0,0,0.247,0.269,0.1640
3343,Indonesia,2017,2.21,0.216,962,1020.0,0,0,0.236,0.280,0.1550


In [54]:
# save dataframe
cleaned_df.to_csv('Data/Processed/save_the_earth_processed_data.csv', index=True)

<div class="alert alert-danger" style="color:black">
    
**Credential**
    
FREE DATA FROM WORLD BANK VIA GAPMINDER.ORG, CC-BY LICENSE
</div>